# Class 4: Transfer Learning
## Overview
In this class, we will explore **Transfer Learning**, a powerful technique in deep learning that leverages pre-trained models to solve new tasks efficiently. We will cover:
- Using pre-trained models like VGG16 and ResNet.
- The difference between **feature extraction** and **fine-tuning**.
- Practical application to an image classification task using a real-world dataset.

This notebook uses TensorFlow/Keras and assumes you have a basic understanding of neural networks.

## Prerequisites
Make sure you have the following libraries installed:
- TensorFlow
- NumPy
- Matplotlib
- Scikit-learn

You can install them using:
```bash
pip install tensorflow numpy matplotlib scikit-learn
```
We will also use the **Cats vs Dogs** dataset, which is available through TensorFlow Datasets.

In [ ]:
# Import necessary libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


## 1. What is Transfer Learning?
Transfer learning involves taking a model trained on a large, general dataset (e.g., ImageNet) and reusing it for a specific task. This is particularly useful when you have limited data for your task.

### Key Concepts:
- **Pre-trained Models**: Models like VGG16, ResNet, or InceptionV3 trained on ImageNet (1.4M images, 1000 classes).
- **Feature Extraction**: Use the pre-trained model as a fixed feature extractor, only training a new classifier on top.
- **Fine-Tuning**: Adjust the weights of the pre-trained model along with training a new classifier for better performance.

## 2. Loading and Preprocessing the Dataset
We will use the **Cats vs Dogs** dataset for binary classification. The images will be resized to match the input size of pre-trained models (224x224 for VGG16/ResNet).

Let's load and preprocess the dataset.

In [ ]:
# Load the Cats vs Dogs dataset
(ds_train, ds_test), ds_info = tfds.load('cats_vs_dogs', split=['train[:80%]', 'train[80%:]'], 
                                         with_info=True, as_supervised=True)

# Define preprocessing function
IMG_SIZE = 224
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize to [0,1]
    return image, label

# Apply preprocessing
BATCH_SIZE = 32
ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Visualize a few samples
def show_images(dataset):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(9):
            plt.subplot(3, 3, i + 1)
            plt.imshow(images[i])
            plt.title('Cat' if labels[i] == 0 else 'Dog')
            plt.axis('off')
    plt.show()

show_images(ds_train)

## 3. Feature Extraction with VGG16
In **feature extraction**, we use a pre-trained model to extract features from the input images and train only a new classifier on top. Here, we'll use VGG16 as the base model.

In [ ]:
# Load VGG16 model (without the top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(ds_train, epochs=5, validation_data=ds_test)

# Plot training results
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## 4. Fine-Tuning with ResNet50
In **fine-tuning**, we unfreeze some layers of the pre-trained model and train them along with the new classifier. This can improve performance but requires careful tuning to avoid overfitting.

Here, we'll use ResNet50 and fine-tune the last few layers.

In [ ]:
# Load ResNet50 model (without the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze all layers initially
base_model.trainable = False

# Unfreeze the last few layers
base_model.trainable = True
fine_tune_at = 100  # Unfreeze from this layer onwards
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Add custom layers on top
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs, outputs)

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(ds_train, epochs=5, validation_data=ds_test)

# Plot training results
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## 5. Practical Application: Evaluating the Model
Let's evaluate the fine-tuned ResNet50 model on the test set and visualize some predictions.

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(ds_test)
print(f'Test accuracy: {test_accuracy:.4f}')

# Visualize predictions
def show_predictions(dataset, model):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        preds = model.predict(images)
        for i in range(9):
            plt.subplot(3, 3, i + 1)
            plt.imshow(images[i])
            pred_label = 'Dog' if preds[i] > 0.5 else 'Cat'
            true_label = 'Dog' if labels[i] == 1 else 'Cat'
            plt.title(f'Pred: {pred_label}\nTrue: {true_label}')
            plt.axis('off')
    plt.show()

show_predictions(ds_test, model)

## 6. Conclusion
In this notebook, we:
- Learned how to use pre-trained models (VGG16, ResNet50) for transfer learning.
- Implemented **feature extraction** by freezing the base model and training a new classifier.
- Performed **fine-tuning** by unfreezing parts of the base model for better performance.
- Applied these techniques to classify images in the Cats vs Dogs dataset.

### Next Steps:
- Experiment with other pre-trained models (e.g., InceptionV3, EfficientNet).
- Try transfer learning on your own dataset.
- Explore advanced fine-tuning strategies, such as learning rate schedules.